# A Recommender System Using >>Apache Spark 2.4
### Predictive Analytics
#### Licence:
You can use this code for anything you may wish only leave this page:
#### AS IS; HOW IS, WHERE IS

In [27]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Recommender System').getOrCreate()

### Reading the Dataset

In [29]:
df = spark.read.csv('./static/petrol_stations.csv', inferSchema=True, header=True)
df.show() # Pulls the fist ten rows of the dataset

+------+---------------+------+
|userId|oil_gas_company|rating|
+------+---------------+------+
|   196|          Shell|     3|
|    63|          Shell|     3|
|   226|          Shell|     5|
|   154|          Shell|     3|
|   306|          Shell|     5|
|   296|          Shell|     4|
|    34|          Shell|     5|
|   271|          Shell|     4|
|   201|          Shell|     4|
|   209|          Shell|     4|
|    35|          Shell|     2|
|   354|          Shell|     5|
|   199|          Shell|     5|
|   113|          Shell|     2|
|     1|          Shell|     5|
|   173|          Shell|     5|
|   360|          Shell|     4|
|   234|          Shell|     4|
|    14|          Shell|     4|
|   309|          Shell|     4|
+------+---------------+------+
only showing top 20 rows



### Data size in terms of rows and records

In [30]:
print((df.count(), len(df.columns)))

(5386, 3)


Our Dataset contains 5386 records with only three columns

Data Schema

In [31]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- oil_gas_company: string (nullable = true)
 |-- rating: integer (nullable = true)



#### Two of the columns are Numerical while oil_gas_company column is categorical

#### Top users by number of companies rated

In [34]:
df.groupBy('userId').count().orderBy('count', ascending=False).show(10, False)

+------+-----+
|userId|count|
+------+-----+
|276   |25   |
|13    |24   |
|378   |24   |
|416   |24   |
|94    |23   |
|450   |22   |
|655   |22   |
|222   |22   |
|796   |21   |
|417   |21   |
+------+-----+
only showing top 10 rows



#### Bottom users by number of companies rated

In [6]:
df.groupBy('userId').count().orderBy('count', ascending=True).show(10, False)

+------+-----+
|userId|count|
+------+-----+
|723   |1    |
|133   |1    |
|688   |1    |
|596   |1    |
|853   |1    |
|300   |1    |
|787   |1    |
|842   |1    |
|587   |1    |
|78    |1    |
+------+-----+
only showing top 10 rows



## Feature Engineering

In [35]:
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, IndexToString

#### So now we create the stringindexer object by mentioning the input column and output column.

In [38]:
stringIndexer = StringIndexer(inputCol='oil_gas_company',outputCol='company_no')
model = stringIndexer.fit(df)
indexed = model.transform(df)
indexed.show(10, False)

+------+---------------+------+----------+
|userId|oil_gas_company|rating|company_no|
+------+---------------+------+----------+
|196   |Shell          |3     |18.0      |
|63    |Shell          |3     |18.0      |
|226   |Shell          |5     |18.0      |
|154   |Shell          |3     |18.0      |
|306   |Shell          |5     |18.0      |
|296   |Shell          |4     |18.0      |
|34    |Shell          |5     |18.0      |
|271   |Shell          |4     |18.0      |
|201   |Shell          |4     |18.0      |
|209   |Shell          |4     |18.0      |
+------+---------------+------+----------+
only showing top 10 rows



In [41]:
print (indexed.groupBy('oil_gas_company','company_no').count().orderBy('count',ascending=False).show(10,False),
       indexed.groupBy('company_no').count().orderBy('count',ascending=False).show(10,False))

+------------------------+----------+-----+
|oil_gas_company         |company_no|count|
+------------------------+----------+-----+
|Obama Oil               |0.0       |452  |
|Pride of Libya          |1.0       |390  |
|Nairobi Oil             |2.0       |365  |
|National Oil of Naigeria|3.0       |303  |
|Total                   |4.0       |297  |
|Twister Oil             |5.0       |293  |
|SA gas                  |6.0       |280  |
|Arrow Gas               |7.0       |254  |
|Oil Ghana               |8.0       |243  |
|Rubis                   |9.0       |227  |
+------------------------+----------+-----+
only showing top 10 rows

+----------+-----+
|company_no|count|
+----------+-----+
|0.0       |452  |
|1.0       |390  |
|2.0       |365  |
|3.0       |303  |
|4.0       |297  |
|5.0       |293  |
|6.0       |280  |
|7.0       |254  |
|8.0       |243  |
|9.0       |227  |
+----------+-----+
only showing top 10 rows

None None


## Spliting The Data Set
We split it into a 80 to 20 ratio to train the model and test its accuracy

In [42]:
train,test=indexed.randomSplit([0.8,0.2])
print("Training set", train.count())
print("Testing set", test.count())

Training set 4266
Testing set 1120


##  Building and Training the Model

In [43]:
from pyspark.ml.recommendation import ALS
rec=ALS(maxIter=10,regParam=0.01,userCol='userId',itemCol='company_no',ratingCol='rating',nonnegative=True)
rec_model = rec.fit(train)

### Performance evaluation on our test data
Here we will chack the performance of our model on unseen data.

In [44]:
predict_ratings = rec_model.transform(test)
predict_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- oil_gas_company: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- company_no: double (nullable = false)
 |-- prediction: float (nullable = false)



In [45]:
predict_ratings.show(10)

+------+---------------+------+----------+----------+
|userId|oil_gas_company|rating|company_no|prediction|
+------+---------------+------+----------+----------+
|   325|   Bluevale gas|     3|      28.0| 2.2565808|
|   429|   Bluevale gas|     3|      28.0| 2.2462227|
|   881|   Bluevale gas|     3|      28.0|  1.980818|
|   268|   Bluevale gas|     3|      28.0| 4.4477963|
|    38|   Bluevale gas|     5|      28.0| 0.9500026|
|   200|   Bluevale gas|     4|      28.0|  3.492417|
|   487|   Bluevale gas|     3|      28.0| 5.2712016|
|   731|    Sabrina gas|     4|      26.0|  2.112349|
|   280|    Sabrina gas|     5|      26.0|  4.223317|
|   340|    Sabrina gas|     4|      26.0| 5.8385525|
+------+---------------+------+----------+----------+
only showing top 10 rows



In [46]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName = 'rmse', predictionCol = 'prediction', labelCol = 'rating')
rmse = evaluator.evaluate(predict_ratings)
print(rmse)

nan


##### The rmse is non we can equate this to zero meaning there is no error.

### Recommend Top Petrol Stations That Active User Might Like
The first step is to create a list of unique companies in the dataframe.

In [47]:
unique_company=indexed.select('company_no').distinct()
unique_company.count() ## Total numbe rof individual companies in our new dataframe

31

In [48]:
a = unique_company.alias('a')

#### We can select any user within the dataset for which we need to recommend other petrol station. In our case, we go ahead with userId = 96.

#### We will filter the movies that this active user has already rated or seen.

In [49]:
user_id=96
rated_companies=indexed.filter(indexed['userId'] == user_id).select('company_no').distinct()
rated_companies.count() # Number of companies the user has rated

7

In [20]:
b=rated_companies.alias('b')

#### So, there are total of 7 unique companies that this active user has already rated. So, we would want to recommend new petrol stations from the remaining sets

In [21]:
total_companies = a.join(b, a.company_no == b.company_no,how='left')
total_companies.show()

+----------+----------+
|company_no|company_no|
+----------+----------+
|       8.0|       8.0|
|       0.0|       0.0|
|       7.0|      null|
|      29.0|      null|
|      18.0|      null|
|       1.0|       1.0|
|      25.0|      null|
|       4.0|      null|
|      23.0|      null|
|      11.0|      11.0|
|      21.0|      null|
|      14.0|      null|
|      22.0|      null|
|       3.0|      null|
|      19.0|      null|
|      28.0|      null|
|       2.0|      null|
|      17.0|      null|
|      27.0|      null|
|      10.0|      null|
+----------+----------+
only showing top 20 rows



In [22]:
remaining_companies=total_companies.where(col("b.company_no").isNull()).select(a.company_no).distinct()
remaining_companies.count()

24

In [23]:
remaining_companies=remaining_companies.withColumn("userId",lit(int(user_id)))
remaining_companies.show(10, False)

+----------+------+
|company_no|userId|
+----------+------+
|7.0       |96    |
|29.0      |96    |
|18.0      |96    |
|25.0      |96    |
|4.0       |96    |
|23.0      |96    |
|21.0      |96    |
|14.0      |96    |
|22.0      |96    |
|3.0       |96    |
+----------+------+
only showing top 10 rows



### Finally, we can now make the predictions on this remaining companies dataset for the active user using the recommender model that we built earlier. We filter only a few top recommendations that have the highest predicted ratings.

In [25]:
recommendations = rec_model.transform(remaining_companies).orderBy('prediction', ascending = False)
recommendations.show(100, False)

+----------+------+----------+
|company_no|userId|prediction|
+----------+------+----------+
|14.0      |96    |4.1148977 |
|18.0      |96    |3.9837985 |
|25.0      |96    |3.7704606 |
|2.0       |96    |3.711454  |
|24.0      |96    |3.5146418 |
|4.0       |96    |3.2447567 |
|21.0      |96    |3.2100048 |
|17.0      |96    |2.9156313 |
|12.0      |96    |2.6432858 |
|3.0       |96    |2.575119  |
|15.0      |96    |2.512583  |
|28.0      |96    |2.3847785 |
|16.0      |96    |2.3674417 |
|10.0      |96    |2.3335602 |
|13.0      |96    |2.328898  |
|5.0       |96    |2.265682  |
|29.0      |96    |2.19795   |
|20.0      |96    |1.9376147 |
|7.0       |96    |1.9060937 |
|23.0      |96    |1.7991966 |
|30.0      |96    |1.719182  |
|27.0      |96    |1.5071282 |
|22.0      |96    |1.415078  |
|19.0      |96    |1.317497  |
+----------+------+----------+



### Let us add the company names to the recommendations

In [26]:
company_name = IndexToString(inputCol="company_no",outputCol="oil_gas_company",labels=model.labels)
final_recommendations=company_name.transform(recommendations)
final_recommendations.show(100,False)

+----------+------+----------+------------------------+
|company_no|userId|prediction|oil_gas_company         |
+----------+------+----------+------------------------+
|14.0      |96    |4.1148977 |Abuja gas               |
|18.0      |96    |3.9837985 |Shell                   |
|25.0      |96    |3.7704606 |Pro gas                 |
|2.0       |96    |3.711454  |Nairobi Oil             |
|24.0      |96    |3.5146418 |Amani gas               |
|4.0       |96    |3.2447567 |Total                   |
|21.0      |96    |3.2100048 |Oillibya                |
|17.0      |96    |2.9156313 |Sahara Pride            |
|12.0      |96    |2.6432858 |K gas                   |
|3.0       |96    |2.575119  |National Oil of Naigeria|
|15.0      |96    |2.512583  |Abuja Oil               |
|28.0      |96    |2.3847785 |Bluevale gas            |
|16.0      |96    |2.3674417 |Vivo                    |
|10.0      |96    |2.3335602 |Dangote Gas             |
|13.0      |96    |2.328898  |Oil Uganda        

### Thats it guys
### A simple collaborative filtering based recommender system in PySpark using the ALS method to recommend gas stations to the users